In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0.9000     ✔ purrr   0.3.4     
✔ tibble  3.0.1          ✔ dplyr   0.8.5     
✔ tidyr   1.0.3          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.5.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



[1] "a6-essemble"

In [2]:
data <- read.csv("../input/a6-essemble/Earthquate_Damage.csv")
cat_cols <- c("land_surface_condition","foundation_type",
              "roof_type","ground_floor_type",
             "other_floor_type","position",
              "plan_configuration","legal_ownership_status")
install.packages("nnet")
library(nnet)
j <- 1
for(i in 1:length(cat_cols)){
    tmp <- class.ind(data[,cat_cols[i]])
    colnames(tmp) <- (j:(j+length(colnames(tmp)) - 1))
    data <- cbind(data,tmp)
    j <- (j+length(colnames(tmp)))
}
data <- data[,!(names(data)%in%cat_cols)]
scaled_data <- scale(data[,1:31],center=TRUE,scale=TRUE)
other_data <- data[,32:70]
data <- cbind(scaled_data,other_data)
rm(scaled_data)
rm(other_data)
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
rm(trn_idx)
rm(val_idx)
train <- rbind(train,val)
rm(val)
perf_eval_multi <- function(cm){
  
  # Simple Accuracy
  ACC = sum(diag(cm))/sum(cm)
  
  # Balanced Correction Rate
  BCR = 1
  for (i in 1:dim(cm)[1]){
    BCR = BCR*(cm[i,i]/sum(cm[i,])) 
  }
  
  BCR = BCR^(1/dim(cm)[1])
  
  return(c(ACC, BCR))
}
prediction_one <- function(pred){
    tmp <- apply(pred,1,which.max)
    for (i in 1:nrow(pred)){
        pred[i,tmp[i]] <- 1
        pred[i,-tmp[i]] <- 0
    }
    return(pred)
}
set.seed(42)
trn_idx <- sample(1:nrow(data), 150000)
train <- data[trn_idx,]
tmp <- data[-trn_idx,]
val_idx <- sample(1:nrow(tmp),50000)
val <- tmp[val_idx,]
test <- tmp[-val_idx,]
x_train <- train[,!(names(train)%in%"damage_grade")]
y_train <- train[,(names(train)%in%"damage_grade")]
y_train <- class.ind(y_train)
x_val <- val[,!(names(val)%in%"damage_grade")]
y_val <- val[,(names(val)%in%"damage_grade")]
y_val <- class.ind(y_val) 
x_test <- test[,!(names(test)%in%"damage_grade")]
y_test <- test[,(names(test)%in%"damage_grade")]
#y_test <- class.ind(y_test)
all_x_train <- rbind(x_train,x_val)
all_y_train <- rbind(y_train,y_val)
all_x_train <- as.matrix(all_x_train)
all_y_train <- as.matrix(all_y_train)
x_test <- as.matrix(x_test)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
library(keras)

In [4]:
ann_boots <- function(seed,nh,epochs){
    
    set.seed(seed)
    boots_idx <- sample(1:nrow(all_x_train),50000,replace=TRUE)
    tmp_x_train <- all_x_train[boots_idx,]
    tmp_y_train <- all_y_train[boots_idx,]
    model <- keras_model_sequential() 
    model %>% 
      layer_dense(units = nh, input_shape = c(69)) %>% 
      layer_activation('relu') %>% 
      layer_dense(units = 3) %>% 
      layer_activation('softmax')
    
    model %>% compile(
      loss = "categorical_crossentropy",
      optimizer = optimizer_rmsprop(),
      metrics = c("accuracy")
    )
    
    history <- model %>% fit(
      tmp_x_train, tmp_y_train, verbose=0,
      epochs = epochs, batch_size = 4096, 
    )
    
    preds <- model %>% predict_proba(x_test)
    return(preds)
}

In [5]:
ann_result <- array(0, dim=c(nrow(test),3,330))

In [6]:
tmp_preds <- c(rep(0,times=nrow(x_test)))
tmp_repeat_preds <- c(rep(0,times=nrow(x_test)))
iter_cnt <- 1
for (i in 1:330){
    ann_result[,,iter_cnt] <- ann_boots(i,50,50)
    iter_cnt <- iter_cnt+1
}

In [7]:
generate_bagging <- function(index,num){
    result <- array(0, dim=c(nrow(test),3))
    for (i in index){
        result <- result + ann_result[,,i]
    }
    result <- result/num
    result <- max.col(result)
    return(result)
}

In [8]:
ann_bagging30 = matrix(0,30,2)
colnames(ann_bagging30) <- c("ACC", "BCR")

In [9]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*10):(30+(i-1)*10))
    tmp_cm <- table(y_test, generate_bagging(index,30))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging30[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [10]:
ann_bagging60 = matrix(0,30,2)
colnames(ann_bagging60) <- c("ACC", "BCR")

In [11]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*9):(60+(i-1)*9))
    tmp_cm <- table(y_test, generate_bagging(index,60))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging60[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [12]:
ann_bagging90 = matrix(0,30,2)
colnames(ann_bagging90) <- c("ACC", "BCR")

In [13]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*8):(90+(i-1)*8))
    tmp_cm <- table(y_test, generate_bagging(index,90))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging90[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [14]:
ann_bagging120 = matrix(0,30,2)
colnames(ann_bagging120) <- c("ACC", "BCR")

In [15]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*7):(120+(i-1)*7))
    tmp_cm <- table(y_test, generate_bagging(index,120))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging120[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [16]:
ann_bagging150 = matrix(0,30,2)
colnames(ann_bagging150) <- c("ACC", "BCR")

In [17]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*6):(150+(i-1)*6))
    tmp_cm <- table(y_test, generate_bagging(index,150))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging150[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [18]:
ann_bagging180 = matrix(0,30,2)
colnames(ann_bagging180) <- c("ACC", "BCR")

In [19]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*5):(180+(i-1)*5))
    tmp_cm <- table(y_test, generate_bagging(index,180))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging180[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [20]:
ann_bagging210 = matrix(0,30,2)
colnames(ann_bagging210) <- c("ACC", "BCR")

In [21]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*4):(210+(i-1)*4))
    tmp_cm <- table(y_test, generate_bagging(index,210))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging210[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [22]:
ann_bagging240 = matrix(0,30,2)
colnames(ann_bagging240) <- c("ACC", "BCR")

In [23]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*3):(240+(i-1)*3))
    tmp_cm <- table(y_test, generate_bagging(index,240))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging240[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [24]:
ann_bagging270 = matrix(0,30,2)
colnames(ann_bagging270) <- c("ACC", "BCR")

In [25]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*2):(270+(i-1)*2))
    tmp_cm <- table(y_test, generate_bagging(index,270))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging270[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [26]:
ann_bagging300 = matrix(0,30,2)
colnames(ann_bagging300) <- c("ACC", "BCR")

In [27]:
iters_cnt <- 1
for (i in 1:30){
    index <- c((1+(i-1)*1):(300+(i-1)*1))
    tmp_cm <- table(y_test, generate_bagging(index,300))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))
    tmp_cm <- cbind(tmp_cm,c(0,0,0))[,1:3]
    ann_bagging300[iters_cnt,1:2] <- perf_eval_multi(tmp_cm)
    iters_cnt <- iters_cnt+1
}

In [28]:
ann_bagging30

ACC,BCR
0.6305837,0.4747559
0.6311612,0.4748665
0.6308807,0.4731470
0.6304516,0.4744132
0.6305176,0.4732387
0.6303691,0.4721659
0.6316067,0.4709047
0.6307652,0.4688672
0.6303361,0.4723218
0.6302371,0.4745329


In [29]:
mean(ann_bagging30[,1])

[1] 0.630362

In [30]:
mean(ann_bagging30[,2])

[1] 0.4754524

In [31]:
sd(ann_bagging30[,1])

[1] 0.0006596522

In [32]:
sd(ann_bagging30[,2])

[1] 0.004502105

In [33]:
ann_bagging60

ACC,BCR
0.6309302,0.4745631
0.6306332,0.4746051
0.6309467,0.4727279
0.6306332,0.4712904
0.6309467,0.4708598
0.6311447,0.4727025
0.6311942,0.4735523
0.6316397,0.4716527
0.6313757,0.4728342
0.6305341,0.4739734


In [34]:
mean(ann_bagging60[,1])

[1] 0.6304879

In [35]:
mean(ann_bagging60[,2])

[1] 0.475444

In [36]:
sd(ann_bagging60[,1])

[1] 0.0005153264

In [37]:
sd(ann_bagging60[,2])

[1] 0.002818946

In [38]:
ann_bagging90

ACC,BCR
0.6305672,0.4727637
0.6309632,0.4729582
0.6310622,0.4734692
0.6307487,0.4714975
0.6310622,0.4734473
0.6309632,0.4722772
0.6307487,0.4732262
0.6310622,0.4736241
0.6309302,0.4726731
0.6305837,0.4736600


In [39]:
mean(ann_bagging90[,1])

[1] 0.6305006

In [40]:
mean(ann_bagging90[,2])

[1] 0.4756278

In [41]:
sd(ann_bagging90[,1])

[1] 0.0003585929

In [42]:
sd(ann_bagging90[,2])

[1] 0.002392969

In [43]:
ann_bagging120

ACC,BCR
0.6305011,0.4732994
0.6308642,0.4731160
0.6307157,0.4728756
0.6307157,0.4731229
0.6309302,0.4731955
0.6306002,0.4737174
0.6307157,0.4744138
0.6305341,0.4749233
0.6307322,0.4754006
0.6308147,0.4749309


In [44]:
mean(ann_baggin120[,1])

ERROR: Error in mean(ann_baggin120[, 1]): object 'ann_baggin120' not found


In [45]:
mean(ann_bagging120[,2])

[1] 0.4756705

In [46]:
sd(ann_bagging120[,1])

[1] 0.000283495

In [47]:
sd(ann_bagging120[,2])

[1] 0.001885836

In [48]:
ann_bagging150

ACC,BCR
0.6304846,0.4729625
0.6306167,0.4734985
0.6306167,0.4742697
0.6307157,0.4747496
0.6304846,0.4743811
0.6306167,0.4752055
0.6305507,0.4744619
0.6304351,0.4739602
0.6303031,0.4738648
0.6305011,0.4734312


In [49]:
mean(ann_baggin150[,1])

ERROR: Error in mean(ann_baggin150[, 1]): object 'ann_baggin150' not found


In [50]:
mean(ann_bagging150[,2])

[1] 0.4758151

In [51]:
sd(ann_bagging150[,1])

[1] 0.0001815281

In [52]:
sd(ann_bagging150[,2])

[1] 0.001836357

In [53]:
ann_bagging180

ACC,BCR
0.6305176,0.4748469
0.6306167,0.4748284
0.6305341,0.4736771
0.6308477,0.4735745
0.6306332,0.4739923
0.6303526,0.4725945
0.6303196,0.4734067
0.6305176,0.4746782
0.6304351,0.4742415
0.6306167,0.4750519


In [54]:
mean(ann_baggin180[,1])

ERROR: Error in mean(ann_baggin180[, 1]): object 'ann_baggin180' not found


In [55]:
mean(ann_bagging180[,2])

[1] 0.4759703

In [56]:
sd(ann_bagging180[,1])

[1] 0.0001045077

In [57]:
sd(ann_bagging180[,2])

[1] 0.001695622

In [58]:
ann_bagging210

ACC,BCR
0.6303526,0.4730373
0.6304186,0.4738127
0.6304351,0.4738292
0.6305672,0.4739318
0.6305341,0.4743414
0.6305176,0.4741623
0.6306167,0.4738901
0.6305341,0.4749813
0.6306002,0.4751975
0.6307322,0.4757832


In [59]:
mean(ann_baggin210[,1])

ERROR: Error in mean(ann_baggin210[, 1]): object 'ann_baggin210' not found


In [60]:
mean(ann_bagging210[,2])

[1] 0.4757727

In [61]:
sd(ann_bagging210[,1])

[1] 0.0001915245

In [62]:
sd(ann_bagging210[,2])

[1] 0.001221208

In [63]:
ann_bagging240

ACC,BCR
0.6306332,0.4751062
0.6307157,0.4752641
0.6308147,0.4755434
0.6305837,0.4751038
0.6305507,0.4751252
0.6305507,0.4750864
0.6305837,0.4756046
0.6308477,0.4759865
0.6308147,0.4759843
0.6306332,0.4758405


In [64]:
mean(ann_baggin240[,1])

ERROR: Error in mean(ann_baggin240[, 1]): object 'ann_baggin240' not found


In [65]:
mean(ann_bagging240[,2])

[1] 0.475922

In [66]:
sd(ann_bagging240[,1])

[1] 0.0001464817

In [67]:
sd(ann_bagging240[,2])

[1] 0.0005074222

In [68]:
ann_bagging270

ACC,BCR
0.6307487,0.4758085
0.6307652,0.4759455
0.6306167,0.4758014
0.6304681,0.4753747
0.6305507,0.4752639
0.6306992,0.4755809
0.6307322,0.4758739
0.6306992,0.4756892
0.6307157,0.4758003
0.6306827,0.4759201


In [69]:
mean(ann_baggin270[,1])

ERROR: Error in mean(ann_baggin270[, 1]): object 'ann_baggin270' not found


In [70]:
mean(ann_bagging270[,2])

[1] 0.4758314

In [71]:
sd(ann_bagging270[,1])

[1] 0.0001317547

In [72]:
sd(ann_bagging270[,2])

[1] 0.0003814019

In [73]:
ann_bagging300

ACC,BCR
0.6307322,0.4757884
0.6306002,0.4757414
0.6305672,0.4754608
0.6306662,0.4755822
0.6305507,0.4753767
0.6305837,0.4754787
0.6306332,0.4754651
0.6306497,0.4756013
0.6305837,0.4755184
0.6305672,0.4756190


In [74]:
mean(ann_baggin300[,1])

ERROR: Error in mean(ann_baggin300[, 1]): object 'ann_baggin300' not found


In [75]:
mean(ann_bagging300[,2])

[1] 0.4758552

In [76]:
sd(ann_bagging300[,1])

[1] 0.000106654

In [77]:
sd(ann_bagging300[,2])

[1] 0.0003161244